In [1]:
import json, random, re, collections, itertools, base64, sys
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
ls = lambda path: print("\n".join(map(str, path.iterdir())))

sys.path.append('../src')
mturk_path = Path("../mturk/results")

%load_ext autoreload
%autoreload 2

In [17]:
from instruction_parsing import program
import annotation_io
import read_data
annotation_io.set_recipe_source("npn-cooking")
from edit_distance import edit_distance


def read_annotation(annotation_id, mturk_batch=None):
    if mturk_batch:
        annotation = annotation_io.mturk_annotation(mturk_batch, annotation_id)
    else:
        annotation = annotation_io.get_annotation(annotation_id)
    return annotation

def annotator_agreement(annotation1,annotation2):
    lst1 = [(x.command.name,x.arg,x.resource) for x in program(annotation1)]
    lst2 = [(x.command.name,x.arg,x.resource) for x in program(annotation2)]
    dst,_ = edit_distance(lst1, lst2)
    return 1-dst/(len(lst1)+len(lst2))


In [3]:
ls(mturk_path)

../mturk/results/4335716.json
../mturk/results/results_approved.csv
../mturk/results/4289268.json
../mturk/results/Batch_4258572_batch_results.csv
../mturk/results/Batch_4286064_batch_results.csv
../mturk/results/Batch_4301520_batch_results.csv
../mturk/results/Batch_4335716_batch_results.csv:Zone.Identifier
../mturk/results/Batch_4289268_batch_results.csv
../mturk/results/Batch_4280306_batch_results.csv
../mturk/results/4286064.json
../mturk/results/4301520.json
../mturk/results/Batch_4335716_batch_results.csv


In [4]:
def load_results(batch):
    df = pd.read_csv(str(next(mturk_path.rglob(f"*{batch}*.csv"))))
    #hack:
    df = df.rename(columns={"Input.v9":"Answer.Title"})
    df = df[[c for c in df.columns if "Answer." in c or 'Id' in c]]
    df = df.rename(columns={c:c.split('.',1)[1] for c in df.columns if '.' in c})
    return df.set_index("AssignmentId")
# batch = 4289268#4286064#4280306
batch = 4335716#4301520#NPN
df=load_results(batch)
df

,HITId,HITTypeId,WorkerId,Title,events,feedback,id,msgbox_count,seconds_spent,status
AssignmentId,,,,,,,,,,
38SKSKU7R7SJQCSLXKHLNCCTCQ4IL5,3GITHABAD3G3G7KAUS77O4DEQW5N2Y,3RL1BYM297F93XL5TMADNASUYZT97G,AOWI7X5AILAHM,cuban_red_beans__rice,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",Tried to discard just bay leafs but it will no...,cuban_red_beans__rice,2,"[19,1,19,27,3,34,1,13,206]",-9
3NL0RFNU0LIEYZBORV3GEBPVI9VK4O,3XUSYT70JYWNYLQJQBPQ9B85FUK0DA,3RL1BYM297F93XL5TMADNASUYZT97G,A2X52OGT1GI4SJ,augustas_chilled_tomato_soup_basil_cream,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},augustas_chilled_tomato_soup_basil_cream,34,"[62,244,224,261,68,42,124,169,292]",1
3TY7ZAOG5LFXZBTN8UGE28ZI1WHK08,3XUSYT70JYWNYLQJQBPQ9B85FUK0DA,3RL1BYM297F93XL5TMADNASUYZT97G,A2GU192S99I6IB,augustas_chilled_tomato_soup_basil_cream,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},augustas_chilled_tomato_soup_basil_cream,12,"[17,139,20,67,68,8,9,9,31]",1
34S9DKFK79KV4CABDFJ3MSA674BYN2,3ICOHX7EOH6SQR4JM60AFTQQF6L0E3,3RL1BYM297F93XL5TMADNASUYZT97G,A2GU192S99I6IB,turkey_kabobs,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},turkey_kabobs,1,"[13,11,6,22,26,9,18]",1
3OCHAWUVGUF5W1ZYB65LBMMTB48XKO,3ICOHX7EOH6SQR4JM60AFTQQF6L0E3,3RL1BYM297F93XL5TMADNASUYZT97G,AO2WNSGOXAX52,turkey_kabobs,[],NICE,turkey_kabobs,0,[],-3
...,...,...,...,...,...,...,...,...,...,...
3QL2OFSM9CDI1996HN1EP1IN4FVCN8,3TZ0XG8CCZFTDKI59ZX3GUVX77H89O,3RL1BYM297F93XL5TMADNASUYZT97G,AWBUSVSQFSLZW,chicken__peas_curried_yogurt,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},chicken__peas_curried_yogurt,8,"[760,257,293,105,207]",1
3TGOYF9913HRITAF0FY0X2WNJC5UU4,3TZ0XG8CCZFTDKI59ZX3GUVX77H89O,3RL1BYM297F93XL5TMADNASUYZT97G,A1YFHRAL9KEOHJ,chicken__peas_curried_yogurt,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},chicken__peas_curried_yogurt,6,"[77,35,55,78,71]",1
3DL65MZB8JAE73WTOOYVZEOEQN3ECU,30Z7M1Q8V3F7I3SA15B3CP6HDPZ8A3,3RL1BYM297F93XL5TMADNASUYZT97G,AOWI7X5AILAHM,suppers_[tuesday]_carrot__ham_soup,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},suppers_[tuesday]_carrot__ham_soup,1,"[43,22,18,12,9,34,8,18]",1


In [5]:
def resources(events):
    return {e["resource"] for schedule in events for e in schedule if  e["resource"]!="SERVE" and "VALID" not in e["resource"]}
def max_tools(events):
    lst = [len({e["action"] for e in schedule if  e["resource"]!="SERVE" and "VALID" not in e["resource"] and e["action"].startswith("T")}) for schedule in events]
    return max(lst) if any(lst) else None
def max_steps(events):
    lst = [len(set(map(lambda s: s["start"],t))) for t in events]
    return max(lst) if any(lst) else None
def has_conditions(events):
    return any({e["action"].startswith("L") for schedule in events for e in schedule if  e["resource"]!="SERVE" and "VALID" not in e["resource"]})
df["events"]=df["events"].apply(json.loads)
df["resources"]=df["events"].apply(resources)
df["resources_len"]=df["resources"].apply(len)
df["max_tools"]=df["events"].apply(max_tools)
df["has_conditions"]=df["events"].apply(has_conditions)
df["max_steps"]=df["events"].apply(max_steps)
df

,HITId,HITTypeId,WorkerId,Title,events,feedback,id,msgbox_count,seconds_spent,status,resources,resources_len,max_tools,has_conditions,max_steps
AssignmentId,,,,,,,,,,,,,,,
38SKSKU7R7SJQCSLXKHLNCCTCQ4IL5,3GITHABAD3G3G7KAUS77O4DEQW5N2Y,3RL1BYM297F93XL5TMADNASUYZT97G,AOWI7X5AILAHM,cuban_red_beans__rice,"[[{'start': '2020-01-01T00:00:00', 'end': '202...",Tried to discard just bay leafs but it will no...,cuban_red_beans__rice,2,"[19,1,19,27,3,34,1,13,206]",-9,"{STOVE_MED, STOVE_HI, FAUCET_OFF}",3,2.0,True,2.0
3NL0RFNU0LIEYZBORV3GEBPVI9VK4O,3XUSYT70JYWNYLQJQBPQ9B85FUK0DA,3RL1BYM297F93XL5TMADNASUYZT97G,A2X52OGT1GI4SJ,augustas_chilled_tomato_soup_basil_cream,"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},augustas_chilled_tomato_soup_basil_cream,34,"[62,244,224,261,68,42,124,169,292]",1,"{APPL_PROCESSOR, APPL_BLEND, COUNTER1, REF_CHIL}",4,3.0,True,3.0
3TY7ZAOG5LFXZBTN8UGE28ZI1WHK08,3XUSYT70JYWNYLQJQBPQ9B85FUK0DA,3RL1BYM297F93XL5TMADNASUYZT97G,A2GU192S99I6IB,augustas_chilled_tomato_soup_basil_cream,"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},augustas_chilled_tomato_soup_basil_cream,12,"[17,139,20,67,68,8,9,9,31]",1,"{APPL_PROCESSOR, COUNTER1, REF_CHIL}",3,2.0,True,3.0
34S9DKFK79KV4CABDFJ3MSA674BYN2,3ICOHX7EOH6SQR4JM60AFTQQF6L0E3,3RL1BYM297F93XL5TMADNASUYZT97G,A2GU192S99I6IB,turkey_kabobs,"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},turkey_kabobs,1,"[13,11,6,22,26,9,18]",1,"{COUNTER1, COUNTER2}",2,NaN,True,2.0
3OCHAWUVGUF5W1ZYB65LBMMTB48XKO,3ICOHX7EOH6SQR4JM60AFTQQF6L0E3,3RL1BYM297F93XL5TMADNASUYZT97G,AO2WNSGOXAX52,turkey_kabobs,[],NICE,turkey_kabobs,0,[],-3,{},0,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3QL2OFSM9CDI1996HN1EP1IN4FVCN8,3TZ0XG8CCZFTDKI59ZX3GUVX77H89O,3RL1BYM297F93XL5TMADNASUYZT97G,AWBUSVSQFSLZW,chicken__peas_curried_yogurt,"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},chicken__peas_curried_yogurt,8,"[760,257,293,105,207]",1,"{STOVE_LOW, COUNTER1, STOVE_MEDHI}",3,3.0,True,3.0
3TGOYF9913HRITAF0FY0X2WNJC5UU4,3TZ0XG8CCZFTDKI59ZX3GUVX77H89O,3RL1BYM297F93XL5TMADNASUYZT97G,A1YFHRAL9KEOHJ,chicken__peas_curried_yogurt,"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},chicken__peas_curried_yogurt,6,"[77,35,55,78,71]",1,"{STOVE_LOW, COUNTER1, STOVE_MEDHI}",3,2.0,True,3.0
3DL65MZB8JAE73WTOOYVZEOEQN3ECU,30Z7M1Q8V3F7I3SA15B3CP6HDPZ8A3,3RL1BYM297F93XL5TMADNASUYZT97G,AOWI7X5AILAHM,suppers_[tuesday]_carrot__ham_soup,"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},suppers_[tuesday]_carrot__ham_soup,1,"[43,22,18,12,9,34,8,18]",1,"{STOVE_MED, STOVE_MEDLOW}",2,2.0,True,2.0


In [6]:
df_approve = df.query("max_steps>2 and max_tools<=4 and resources_len>1 and has_conditions")
print("{a}/{t}".format(a=df_approve.shape[0],t=df.shape[0]))

129/257


In [7]:
 status_map = {
 "1": "All OK",
 "0": "All OK, not finished",
 "-1": "Missing Tool",
 "-2": "Missing Resource",
 "-3": "Missing Ingredient",
 "-4": "Redundant unused ingredient",
 "-5": "Missing Time Boundry",
 "-6": "Activity",
"-9": "Validation Error",
 "-10": "Other issue",
 }

In [8]:
html=""
for assignment, row in df.iterrows():
    approved = assignment in df_approve.index
    color = "green" if approved else "red"
    worker=row["WorkerId"]
    title=row["Title"]
    idx = row["id"]
    status = status_map[str(row["status"])]
    display_id=worker#assignment
    html+=f'(worker={worker}) <font color="{color}">{display_id}</font> <a href="http://127.0.0.1:8080/annotate/{batch}/{assignment}" target="_new">({idx}) {title} - {status}</a> '
    if int(row["status"])<0:
        html+=row["feedback"]
    html+='<br />'
display_html(html)

In [9]:
def approve_assignemnts(batch, assignment_ids,workers=[]):
    df = pd.read_csv(str(next(mturk_path.rglob(f"*{batch}*.csv"))))
    #df = df[[c for c in df.columns if "Input." not in c and 'Answer.' not in c]]
    df.loc[df["AssignmentId"].isin(assignment_ids)|df["WorkerId"].isin(workers), 'Approve']='X'
    df.loc[~(df["AssignmentId"].isin(assignment_ids)|df["WorkerId"].isin(workers)), 'Reject']='X'
    df.to_csv(str(mturk_path/f"results_approved.csv"), index=False)
    print ("{l} assignments approved".format(l=(df["Approve"]=='X').sum()))
approve_assignemnts(batch, df_approve.index)

129 assignments approved


In [10]:
approve_assignemnts(batch, df_approve.index,{"A3E21OXYGLFZTB", "A28HQQPBN2MUNR", "A6YDC07DBLRY2", "A2WYCY1FMQOD5F", "A8D0400E5EK2K"})

133 assignments approved


In [11]:
with (mturk_path/f"{batch}.json").open('w') as f:
    groupped= df.query("max_steps>2 and max_tools<=4 and resources_len>1 and has_conditions")\
        .reset_index().groupby("id")["AssignmentId"].apply(list).to_dict()
    json.dump(groupped, f)
groupped

{'all_in_one_casserole': ['3SKRO2GZ77MX60E54DKEGDIS6PJK19'],
 'au_gratin_beef_spinach': ['33NF62TLXPX41HU9U2QKKDEE050JKR',
  '36AHBNMV1X7MI0JYOL50QNYCT04DYU',
  '39U1BHVTDRMSJMI7LQCP5XKLNLNT3D'],
 'augustas_chilled_tomato_soup_basil_cream': ['3NL0RFNU0LIEYZBORV3GEBPVI9VK4O',
  '3TY7ZAOG5LFXZBTN8UGE28ZI1WHK08'],
 'baked_chicken_tomato_sauce': ['308XBLVESOZK629SWUM0N1HFNSMRB8',
  '324G5B4FB9394WM3P0R5Y3UWVBP70V',
  '3C8HJ7UOPDPPRKZGX06MNKU888VZMK'],
 'baked_zucchini_tomatoes': ['3AQF3RZ55EDHX0NO985YVWH463AF6Q',
  '3VW04L3ZLZ1BG1Y5APKA1GUV7B8XXU'],
 'basil_tomatoes': ['31IBVUNM9YU2CQ2PAGD67OX3NWKVFG',
  '3KRVW3HTZTGSQ8D228CERRX5LSAMSF'],
 'beef_bean_chili': ['3IQ1VMJRYZF9JS8EWZMI15XFZPMA92',
  '3M81GAB8A6EKU1USJFX0IB7AR67QBT'],
 'beef_stuffed_peppers': ['3C44YUNSI7KS9MRXXY9HDW50PV3DP2'],
 'black_bean_soup_sour_cream_mango_salsa': ['3JZQSN0I3W5JUTCULLEK7NUT0H9FGO'],
 'boiled_beef_pizzaiola': ['3F1567XTN20168FVLVJ4R5CK36A9Q8',
  '3KMS4QQVK8LOWF8DBCZZ5FMM7YQKFW',
  '3ZAZR5XV07DCIYNPKBC8KOPWE

In [12]:
events=json.loads("""
 [[{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"INNbMITPe:c422149b-8965-4218-b0be-14aa43fba830","action":"INNbMITPe","resource":"VALID_UNUSED","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"I5dODtwdf:a4c50b86-03f6-4040-0361-e55e369b30db","action":"I5dODtwdf","resource":"VALID_UNUSED","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"I10_d4pRP:527dff47-59fc-878a-a08c-d626ae6ad645","action":"I10_d4pRP","resource":"VALID_UNUSED","text":"butter","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"II9-0pj_o:ff5cf91b-0913-0ebd-7365-6af1d4c66221","action":"II9-0pj_o","resource":"VALID_UNUSED","text":"sliced mushrooms","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"I1IusNkrV:0a50aeae-cb90-4e46-ad6c-e02ffb6e3e87","action":"I1IusNkrV","resource":"VALID_UNUSED","text":"celery","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"Iz0wiMjVJ:bc263ebc-32c1-370a-6477-cb7414d7a0c3","action":"Iz0wiMjVJ","resource":"VALID_UNUSED","text":"onion","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"IkMjQzEsw:c36bfc89-93a6-0d69-b3c3-301be7f52c2a","action":"IkMjQzEsw","resource":"VALID_UNUSED","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"II63VkVcC:9cbe6c14-e72c-b20e-6671-b27bce0338c4","action":"II63VkVcC","resource":"VALID_UNUSED","text":"poultry seasoning","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"TPAN:d9445271-d559-a4f0-1c0b-1a938e645471","action":"TPAN","resource":"COUNTER1","text":"Sauce Pan","barColor":"#ff0000"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"INNbMITPe:cb44b949-fd34-9018-3d6d-494af775e564","action":"INNbMITPe","resource":"COUNTER1","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I5dODtwdf:b8c9d119-81b1-afaa-46c0-ec9dbd814d80","action":"I5dODtwdf","resource":"COUNTER1","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"INNbMITPe:fe457281-3e46-2444-4225-653f8213b979","action":"INNbMITPe","resource":"VALID_MIX1","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I5dODtwdf:6168d300-13ee-2953-dd3a-8cc830a4ef45","action":"I5dODtwdf","resource":"VALID_MIX1","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"INNbMITPe:4fa7933c-f354-390f-4f21-a0c94edb1888","action":"INNbMITPe","resource":"VALID_MIX1","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I5dODtwdf:4508fd22-0376-39b3-6b27-0e277106f4bb","action":"I5dODtwdf","resource":"VALID_MIX1","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"TPAN:b17bb402-b636-54d0-420f-e4e1ccfb0668","action":"TPAN","resource":"STOVE_HI","text":"Sauce Pan","barColor":"#ff0000"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I10_d4pRP:bfdcbeee-826b-0ce6-e3ff-1dd480d8fe2e","action":"I10_d4pRP","resource":"VALID_UNUSED","text":"butter","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"II9-0pj_o:94fe17a2-8909-70b9-d7ac-17a12a26c4d3","action":"II9-0pj_o","resource":"VALID_UNUSED","text":"sliced mushrooms","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I1IusNkrV:3bd3af38-2c06-6df0-c495-c828e1eb9006","action":"I1IusNkrV","resource":"VALID_UNUSED","text":"celery","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"Iz0wiMjVJ:4f64dabe-9a6e-0131-98c8-2734e78284ca","action":"Iz0wiMjVJ","resource":"VALID_UNUSED","text":"onion","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"IkMjQzEsw:f460610a-5184-89cb-be03-26d6f478d3f4","action":"IkMjQzEsw","resource":"VALID_UNUSED","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"II63VkVcC:5b3d9e5f-847b-eae9-a81a-d1e43a250a8b","action":"II63VkVcC","resource":"VALID_UNUSED","text":"poultry seasoning","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"TPAN:2effc7e4-557a-8849-5ee5-5fc21aab71fb","action":"TPAN","resource":"STOVE_LOW","text":"Sauce Pan","barColor":"#ff0000"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"LTIME:30db82a6-6b2e-eed2-0ec9-d4390865eb4a","action":"LTIME","resource":"STOVE_LOW","text":"For X minutes","barColor":"#000000"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I10_d4pRP:3364215e-5be0-6212-94d5-61a99f9e1455","action":"I10_d4pRP","resource":"VALID_UNUSED","text":"butter","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"II9-0pj_o:1a5a71ce-bc19-877b-296b-d00944f79290","action":"II9-0pj_o","resource":"VALID_UNUSED","text":"sliced mushrooms","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I1IusNkrV:4d226a8f-ed04-25a9-057e-2b12dfca68ef","action":"I1IusNkrV","resource":"VALID_UNUSED","text":"celery","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"Iz0wiMjVJ:d18be14d-0be2-af01-549b-8c919d318c03","action":"Iz0wiMjVJ","resource":"VALID_UNUSED","text":"onion","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"IkMjQzEsw:cf9bb830-d1ac-2f99-cca6-cccd04d03b17","action":"IkMjQzEsw","resource":"VALID_UNUSED","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"II63VkVcC:e895ef97-6e12-a33e-8402-93d20a3cf174","action":"II63VkVcC","resource":"VALID_UNUSED","text":"poultry seasoning","barColor":"#00ff00"}],[{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"INNbMITPe:4fa7933c-f354-390f-4f21-a0c94edb1888","action":"INNbMITPe","resource":"VALID_MIX1","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"I5dODtwdf:4508fd22-0376-39b3-6b27-0e277106f4bb","action":"I5dODtwdf","resource":"VALID_MIX1","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"TPAN:2effc7e4-557a-8849-5ee5-5fc21aab71fb","action":"TPAN","resource":"STOVE_LOW","text":"Sauce Pan","barColor":"#ff0000"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"I10_d4pRP:3364215e-5be0-6212-94d5-61a99f9e1455","action":"I10_d4pRP","resource":"VALID_UNUSED","text":"butter","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"II9-0pj_o:1a5a71ce-bc19-877b-296b-d00944f79290","action":"II9-0pj_o","resource":"VALID_UNUSED","text":"sliced mushrooms","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"I1IusNkrV:4d226a8f-ed04-25a9-057e-2b12dfca68ef","action":"I1IusNkrV","resource":"VALID_UNUSED","text":"celery","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"Iz0wiMjVJ:d18be14d-0be2-af01-549b-8c919d318c03","action":"Iz0wiMjVJ","resource":"VALID_UNUSED","text":"onion","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"IkMjQzEsw:cf9bb830-d1ac-2f99-cca6-cccd04d03b17","action":"IkMjQzEsw","resource":"VALID_UNUSED","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"II63VkVcC:e895ef97-6e12-a33e-8402-93d20a3cf174","action":"II63VkVcC","resource":"VALID_UNUSED","text":"poultry seasoning","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"TSKILLET:fdb60ed1-7b1a-fdb1-5a5f-9eb120e893bc","action":"TSKILLET","resource":"COUNTER1","text":"Skillet","barColor":"#ff0000"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"INNbMITPe:a6ed59c9-9ab8-7410-15fc-7bfb5fd24249","action":"INNbMITPe","resource":"COUNTER1","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I5dODtwdf:18770646-4184-fd2f-f884-748d88486fd8","action":"I5dODtwdf","resource":"COUNTER1","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I10_d4pRP:71e9a2d5-146e-65fd-9697-499483da9db6","action":"I10_d4pRP","resource":"COUNTER1","text":"butter","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"LMELT:c2dbc791-4694-6bf0-6dde-f84f68c308b4","action":"LMELT","resource":"COUNTER1","text":"Until melted","barColor":"#000000"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"II9-0pj_o:0bdf63c8-5ac3-e5ce-fc52-66b7ef944076","action":"II9-0pj_o","resource":"VALID_UNUSED","text":"sliced mushrooms","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I1IusNkrV:fa630866-abb3-5799-0e5a-396fafe4e675","action":"I1IusNkrV","resource":"VALID_UNUSED","text":"celery","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"Iz0wiMjVJ:8266bd29-bdf9-c991-3d43-27db413ceaa7","action":"Iz0wiMjVJ","resource":"VALID_UNUSED","text":"onion","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"IkMjQzEsw:d7182844-13f1-83e6-a9b7-baa0ed74952e","action":"IkMjQzEsw","resource":"VALID_UNUSED","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"II63VkVcC:1c47a9b0-f859-8a0e-d5d9-5d5b4bfaae73","action":"II63VkVcC","resource":"VALID_UNUSED","text":"poultry seasoning","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"TSKILLET:cc597376-73fd-6751-2751-88ab11dd1c5e","action":"TSKILLET","resource":"COUNTER1","text":"Skillet","barColor":"#ff0000"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"INNbMITPe:fea295cc-2908-c8aa-1a04-8309b5e9373a","action":"INNbMITPe","resource":"COUNTER1","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I5dODtwdf:db02ee02-de03-39ae-29b7-050a57e850a9","action":"I5dODtwdf","resource":"COUNTER1","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I10_d4pRP:c216e0af-cb82-122d-fdc7-cee128008720","action":"I10_d4pRP","resource":"COUNTER1","text":"butter","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"II9-0pj_o:6beafe5e-356a-5aa4-9b2e-7102bb44d2ea","action":"II9-0pj_o","resource":"COUNTER1","text":"sliced mushrooms","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I1IusNkrV:3039ffb9-5ebc-044a-1f14-9eff7a8f9837","action":"I1IusNkrV","resource":"COUNTER1","text":"celery","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"Iz0wiMjVJ:3d3e8bb1-dd1e-d735-b41a-5a0f34f71ffc","action":"Iz0wiMjVJ","resource":"COUNTER1","text":"onion","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"LTIME:f2cb3ae8-a429-fa3e-45c1-6eeb2cc4bd4e","action":"LTIME","resource":"COUNTER1","text":"For X minutes","barColor":"#000000"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"IkMjQzEsw:d39dd4c7-7ef5-1948-008b-efa1edce362e","action":"IkMjQzEsw","resource":"VALID_UNUSED","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"II63VkVcC:6b0606ef-1acb-46db-47a1-b08cea6bb079","action":"II63VkVcC","resource":"VALID_UNUSED","text":"poultry seasoning","barColor":"#00ff00"}],[{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"INNbMITPe:e6038993-8287-6f07-3b93-5c227555fbe8","action":"INNbMITPe","resource":"VALID_MIX1","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I5dODtwdf:7cbe4255-8d16-c02e-ae43-683d5925d550","action":"I5dODtwdf","resource":"VALID_MIX1","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I10_d4pRP:6d6b480a-a23a-a43d-19e1-063c848f02a1","action":"I10_d4pRP","resource":"VALID_MIX1","text":"butter","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"II9-0pj_o:f1d546f0-579b-49a6-9e48-01f89e195ac7","action":"II9-0pj_o","resource":"VALID_MIX1","text":"sliced mushrooms","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I1IusNkrV:ba7b0c7b-2f03-56b1-cd21-ec005c0522e2","action":"I1IusNkrV","resource":"VALID_MIX1","text":"celery","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"Iz0wiMjVJ:732512e8-9de2-562d-546f-3f2c888fc483","action":"Iz0wiMjVJ","resource":"VALID_MIX1","text":"onion","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"IkMjQzEsw:92a1c6ed-1b01-4701-bf39-844426a0373f","action":"IkMjQzEsw","resource":"VALID_MIX1","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"II63VkVcC:2b9f6f4c-822c-3fd0-7159-d742619da587","action":"II63VkVcC","resource":"VALID_MIX1","text":"poultry seasoning","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"INNbMITPe:324301cc-99b2-ecdc-2d6c-2be041f4f36f","action":"INNbMITPe","resource":"VALID_MIX1","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I5dODtwdf:a7619dd7-8843-a709-d456-77d4f47e7d8c","action":"I5dODtwdf","resource":"VALID_MIX1","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I10_d4pRP:77d61ae6-205a-4946-4069-554a832ecb39","action":"I10_d4pRP","resource":"VALID_MIX1","text":"butter","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"II9-0pj_o:3b7fed14-2a29-e326-a1f9-468e96fe2b9c","action":"II9-0pj_o","resource":"VALID_MIX1","text":"sliced mushrooms","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I1IusNkrV:f8513c4b-90f4-e717-6a13-30c26cf193ae","action":"I1IusNkrV","resource":"VALID_MIX1","text":"celery","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"Iz0wiMjVJ:18c54633-7a4b-8b59-ea3d-0842e37f3bca","action":"Iz0wiMjVJ","resource":"VALID_MIX1","text":"onion","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"IkMjQzEsw:60e99402-7e76-ad97-b1c1-51798070a0b7","action":"IkMjQzEsw","resource":"VALID_MIX1","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"II63VkVcC:b493a814-57b8-885c-8fc8-93c89dce69aa","action":"II63VkVcC","resource":"VALID_MIX1","text":"poultry seasoning","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"TSKILLET:cc597376-73fd-6751-2751-88ab11dd1c5e","action":"TSKILLET","resource":"COUNTER1","text":"Skillet","barColor":"#ff0000"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"INNbMITPe:fea295cc-2908-c8aa-1a04-8309b5e9373a","action":"INNbMITPe","resource":"COUNTER1","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"I5dODtwdf:db02ee02-de03-39ae-29b7-050a57e850a9","action":"I5dODtwdf","resource":"COUNTER1","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"I10_d4pRP:c216e0af-cb82-122d-fdc7-cee128008720","action":"I10_d4pRP","resource":"COUNTER1","text":"butter","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"II9-0pj_o:6beafe5e-356a-5aa4-9b2e-7102bb44d2ea","action":"II9-0pj_o","resource":"COUNTER1","text":"sliced mushrooms","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"I1IusNkrV:3039ffb9-5ebc-044a-1f14-9eff7a8f9837","action":"I1IusNkrV","resource":"COUNTER1","text":"celery","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"Iz0wiMjVJ:3d3e8bb1-dd1e-d735-b41a-5a0f34f71ffc","action":"Iz0wiMjVJ","resource":"COUNTER1","text":"onion","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"IkMjQzEsw:d39dd4c7-7ef5-1948-008b-efa1edce362e","action":"IkMjQzEsw","resource":"VALID_UNUSED","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-01T00:00:00","end":"2020-01-02T00:00:00","id":"II63VkVcC:6b0606ef-1acb-46db-47a1-b08cea6bb079","action":"II63VkVcC","resource":"VALID_UNUSED","text":"poultry seasoning","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"TBOWL:db8a2fe2-68aa-fb0f-22b3-214297651b9b","action":"TBOWL","resource":"COUNTER1","text":"Bowl","barColor":"#ff0000"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"IkMjQzEsw:137144c5-95a8-895e-a764-0a0dd0832076","action":"IkMjQzEsw","resource":"COUNTER1","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"II63VkVcC:60569e58-6002-5d6f-e195-8de43f552c3d","action":"II63VkVcC","resource":"COUNTER1","text":"poultry seasoning","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"INNbMITPe:f0a384e0-d731-cb7c-f59f-a71ab8a09d38","action":"INNbMITPe","resource":"COUNTER2","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"I5dODtwdf:e1cbdb10-f8f2-a1c6-7b93-11d1eaa76805","action":"I5dODtwdf","resource":"COUNTER2","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-02T00:00:00","end":"2020-01-03T00:00:00","id":"TBOWL:aecefc36-dcbc-e795-7e1c-4c428d84009d","action":"TBOWL","resource":"OVEN_LOW","text":"Bowl","barColor":"#ff0000"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"TBOWL:21e19bd4-7671-057d-d991-745f84d775b3","action":"TBOWL","resource":"SERVE","text":"Bowl","barColor":"#ff0000"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"TSKILLET:5859ef82-3e0f-96dc-6dd4-9d245a8b4631","action":"TSKILLET","resource":"SERVE","text":"Skillet","barColor":"#ff0000"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"INNbMITPe:04c9ef0c-fd9c-3bfb-033b-c05b74ba539b","action":"INNbMITPe","resource":"SERVE","text":"chicken broth","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I5dODtwdf:bbab6efb-8e29-34ca-2fb3-4ac7d6dabf24","action":"I5dODtwdf","resource":"SERVE","text":"wild rice","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I10_d4pRP:f4116a7d-8937-e1c2-5f19-7a7372d193bc","action":"I10_d4pRP","resource":"SERVE","text":"butter","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"II9-0pj_o:6c94c3f9-bd1f-fb89-4cfa-38e08c581efa","action":"II9-0pj_o","resource":"SERVE","text":"sliced mushrooms","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"I1IusNkrV:99b5dd6d-99c6-a2e5-255f-fdbeb6edbd21","action":"I1IusNkrV","resource":"SERVE","text":"celery","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"Iz0wiMjVJ:7cccc219-a539-78e5-d968-46621d57b2de","action":"Iz0wiMjVJ","resource":"SERVE","text":"onion","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"IkMjQzEsw:ee935b4c-5fe8-d6b7-afd3-5d362b032cca","action":"IkMjQzEsw","resource":"SERVE","text":"corn bread stuffing mix","barColor":"#00ff00"},{"start":"2020-01-03T00:00:00","end":"2020-01-04T00:00:00","id":"II63VkVcC:aedbd89d-677d-a879-16c6-5ae4d3d15a6a","action":"II63VkVcC","resource":"SERVE","text":"poultry seasoning","barColor":"#00ff00"}]]
""")

In [13]:
has_conditions(events)

True

In [14]:
max_steps(events)

3

In [15]:
resources(events)

{'COUNTER1', 'COUNTER2', 'OVEN_LOW', 'STOVE_HI', 'STOVE_LOW'}

In [22]:
errs = collections.defaultdict(int)

html = []
annotations = df.reset_index().groupby("id")["AssignmentId"].apply(list).to_dict()
for recipe_id in tqdm(annotations):
    html.append(f"<h2>{recipe_id}</h2>")
    for p1,p2 in itertools.combinations(annotations[recipe_id],2):
        try:
            sim = annotator_agreement(read_annotation(p1, batch),read_annotation(p2, batch))
        except Exception as e:
            errs[str(e)]+=1
            continue
        line = f'<a href="http://127.0.0.1:8080/annotate/{batch}/{p1}" target="_new">{p1}</a>'
        line += ' ({s:0.2f}) '.format(s=sim)
        line += f'<a href="http://127.0.0.1:8080/annotate/{batch}/{p2}" target="_new">{p2}</a>'
        html.append(line)
html = ("<br />".join(html))
display_html(html)

{'instructions': ['cook sausage over low heat 5-8 mins .', 'add onion , green pepper , and garlic ;', 'saute until tender .', 'drain if necessary .', 'add beans , tomatoes , and seasonings ;', 'simmer , uncovered , 20 mins .', 'serve over rice .'], 'ingredients': ['bean', 'garlic', 'green_pepper', 'onion', 'oregano', 'pepper', 'rice', 'sausage', 'tomato'], 'normalized_ingredients': {'IRghrxgvV': 'bean', 'IzPYYTv5e': 'garlic', 'IatP_Y6-h': 'green_pepper', 'Iz0wiMjVJ': 'onion', 'I5IP2D1SP': 'oregano', 'Is_lS1dmt': 'pepper', 'IN2e0UIJI': 'rice', 'Ii0M2cCWP': 'sausage', 'IAG-HiS9H': 'tomato'}, 'title': 'Easy Red Beans  Rice', 'status': -5, 'validations': [['Ii0M2cCWP'], ['Is_lS1dmt', 'Iz0wiMjVJ', 'IatP_Y6-h', 'IzPYYTv5e'], ['Is_lS1dmt', 'Iz0wiMjVJ', 'IatP_Y6-h', 'IzPYYTv5e'], ['Is_lS1dmt', 'Iz0wiMjVJ', 'IatP_Y6-h', 'IzPYYTv5e'], ['IRghrxgvV', 'IAG-HiS9H'], ['IRghrxgvV', 'IAG-HiS9H'], ['IN2e0UIJI']], 'photo_url': '', 'url': '', 'labels': [], 'feedback': "I watched the tutorial and gray I'm 

In [23]:
print (len(errs))

3
